In [ ]:
from PIL import Image, ImageDraw, ImageFont
 

In [ ]:
class TextFont:
    

In [1]:
# 指定要使用的字体和大小；/Library/Fonts/是macOS字体目录；Linux的字体目录是/usr/share/fonts/
font = ImageFont.truetype('simhei.ttf', 100)
 
# image: 图片   text：要添加的文本  font：字体
def add_text_to_image(image, text, font=font):
    rgba_image = image.convert('RGBA')
    text_overlay = Image.new('RGBA', rgba_image.size, (255, 255, 255, 0))
    image_draw = ImageDraw.Draw(text_overlay)
 
    text_size_x, text_size_y = image_draw.textsize(text, font=font)
    
    for pos_x in range(0, rgba_image.size[0], int(text_size_x )):
        for pos_y in range(0, rgba_image.size[1], int(text_size_y * 1.5)):
            image_draw.text((pos_x, pos_y), text, font=font, fill = (50, 50, 50, 50))
    image_with_text = Image.alpha_composite(rgba_image, text_overlay)
    return image_with_text

<PIL.Image.Image image mode=RGBA size=1500x1500 at 0x2CC8105EFD0>


In [ ]:
im_before = Image.open("./kb-0.jpg")
im_before.show()
im_after = add_text_to_image(im_before, '版权所有：迪英加科技有限公司', font)
im_after.show()
# im.save('im_after.jpg')